# Chapter 18. With, match, and else blocks

Example 18-1. Demonstration of a file object as a context manager

In [2]:
with open('mirror.py') as fp:
    src = fp.read(60)

In [3]:
src

''

In [4]:
len(src)

0

In [5]:
fp

<_io.TextIOWrapper name='mirror.py' mode='r' encoding='UTF-8'>

In [6]:
fp.closed, fp.encoding

(True, 'UTF-8')

In [7]:
fp.read(60)

ValueError: I/O operation on closed file.

Example 18-2. Test-driving the LookingGlass context manager class

In [1]:
from mirror import LookingGlass

In [9]:
with LookingGlass() as what:
    print('Alice, Kitty and Snowdrop')
    print(what)

pordwonS dna yttiK ,ecilA
YKCOWREBBAJ


In [10]:
what

'JABBERWOCKY'

In [11]:
print('back to normal')

back to normal


Example 18-4. Exercising LookingGlass without a with block

In [16]:
manager = LookingGlass()

In [17]:
manager

In [23]:
monster = manager.__enter__()

In [24]:
monster

'JABBERWOCKY'

In [25]:
monster == 'JABBERWOCKY'

True

In [26]:
manager.__exit__(None, None, None)

In [27]:
monster

'JABBERWOCKY'

Example 18-6. Test-driving the looking_glass context manager function

In [2]:
from mirror import looking_glass

with looking_glass() as what:
    print('Jalil va Paria ichdi buyun su')
    print(what)

us nuyub idhci airaP av lilaJ
YKCOWREBBAJ


In [3]:
what

'JABBERWOCKY'

In [4]:
@looking_glass()
def verse():
    print('Alivi ver mana')

In [5]:
verse()

anam rev ivilA


In [6]:
print('bir iki')

bir iki
